#Convert the XML to CSV and makes it importer ready 

In [ ]:
import xml.etree.cElementTree as et
import pandas as pd
import os
import fnmatch
import sqlalchemy as sql
import numpy as np
dbHost='mxnunx41'
dbPort='1521'
dbUser='lims'
dbPassword='Mumwit63'
dbName='hwi1'
dburl='oracle://'
colsOrder=['Sample.Sample_Number', 'Test.Analysis','Result.Name', 'Result.Entry', 'Result.Entered_By','Result.Instrument']

In [87]:
# get Data from DB
def getDataFromDB(sampleNumbersList):
    dburl = 'oracle://'+dbUser+':'+dbPassword+'@'+dbHost+':'+dbPort+'/'+dbName
    engine=sql.create_engine(dburl)
    resultdf=pd.DataFrame()
    sampleNumbrStr=(', '.join("'" + str(item) + "'" for item in sampleNumbersList))
    queryText="SELECT sample_number, analysis, REPLICATE_COUNT,TEST_NUMBER FROM test"
    queryText=queryText+" WHERE SAMPLE_NUMBER IN ("+sampleNumbrStr+") AND analysis IN (SELECT NAME FROM ANALYSIS WHERE ALIAS_NAME LIKE '%VIDAS%')"
    conn = engine.connect()
    resultdf = pd.read_sql(queryText, engine)
    conn.close()
    return resultdf

In [88]:
def generateResultname(testIdentifier,variable):
    if(variable=='R2'):
        return str(testIdentifier)
    else:
        return str('OD'+str(testIdentifier))

In [89]:
# get Data from DB
def getDataFromDB(sampleNumbersList):
    dburl = 'oracle://'+dbUser+':'+dbPassword+'@'+dbHost+':'+dbPort+'/'+dbName
    engine=sql.create_engine(dburl)
    resultdf=pd.DataFrame()
    sampleNumbrStr=(', '.join("'" + str(item) + "'" for item in sampleNumbersList))
    queryText="SELECT sample_number, analysis, REPLICATE_COUNT,TEST_NUMBER FROM test"
    queryText=queryText+" WHERE SAMPLE_NUMBER IN ("+sampleNumbrStr+") AND analysis IN (SELECT NAME FROM ANALYSIS WHERE ALIAS_NAME LIKE '%VIDAS%')"
    conn = engine.connect()
    resultdf = pd.read_sql(queryText, engine)
    conn.close()
    return resultdf

In [90]:
path='C:\\GitHub\\Instrument-Interface\\scripts\\Development\\JupyterNotebooks\\SampleFiles_Instruments\\VIDAS\\'
valList=['value','operatorName','testName','testIdentifier','specimenIdentifier','patientIdentifier','instrumentName']
ListDictOne=[]
files = os.listdir(path)
for filename in files:
    if fnmatch.fnmatch(filename,'*.XML'):
        tree = et.parse(path+filename)
        root = tree.getroot()
        resultDict={}
        for child in root.iter():
            if child.tag in valList:
                resultDict[child.tag]=child.text
        resultDict['filename']=filename
        ListDictOne.append(resultDict)

In [91]:
raw_data=pd.DataFrame(ListDictOne)

In [92]:
raw_data.head()

,filename,instrumentName,operatorName,patientIdentifier,specimenIdentifier,testIdentifier,testName,value
0,MINVIDAS1_190428_191240_100.xml,MINVIDAS1,minn-vidas1,387,falA618117,SLM,Salmonella,0.03|NEGATIVE
1,MINVIDAS1_190428_191240_101.xml,MINVIDAS1,minn-vidas1,387,famA618118,SLM,Salmonella,0.03|NEGATIVE
2,MINVIDAS1_190428_191240_102.xml,MINVIDAS1,minn-vidas1,387,fanA618120,SLM,Salmonella,0.03|NEGATIVE
3,MINVIDAS1_190428_191240_103.xml,MINVIDAS1,minn-vidas1,387,faoA618122,SLM,Salmonella,0.04|NEGATIVE
4,MINVIDAS1_190428_191240_104.xml,MINVIDAS1,minn-vidas1,387,fapA618124,SLM,Salmonella,0.03|NEGATIVE


In [93]:
raw_data["sample_number"] = raw_data["patientIdentifier"].map(str) + raw_data["specimenIdentifier"].str[-6:]

In [94]:
raw_data11 = raw_data["value"].str.split("|", n = 1, expand = True)
raw_data["R1"]= raw_data11[0] 
raw_data["R2"]= raw_data11[1]
raw_data.drop(columns =["value"], inplace = True) 
dfResult = pd.melt(raw_data, id_vars=['filename','instrumentName','operatorName','patientIdentifier','specimenIdentifier','testIdentifier','testName','sample_number'], value_vars=['R1', 'R2'])

In [95]:
#sampleNumbersList=list(raw_data1['sample_number'].unique())
#databasedf=getDataFromDB(sampleNumbersList)
#raw_data1=raw_data1.astype({"sample_number": int})
#dfResult = raw_data1.merge(databasedf, how = 'inner', on = ['sample_number'])
dfResult = dfResult.sort_values(by=['sample_number'])

In [96]:
dfResult['Result.Name'] = dfResult.apply(lambda x: generateResultname(x.testIdentifier,x.variable),axis=1)

In [97]:
dfResult

,filename,instrumentName,operatorName,patientIdentifier,specimenIdentifier,testIdentifier,testName,sample_number,variable,value,Result.Name
612,MINVIDAS3_190429_130746_50.xml,MINVIDAS3,minn-vidas3,387,cbyA589896,SLM,Salmonella,387589896,R2,NEGATIVE,SLM
299,MINVIDAS3_190429_130746_50.xml,MINVIDAS3,minn-vidas3,387,cbyA589896,SLM,Salmonella,387589896,R1,0.03,ODSLM
613,MINVIDAS3_190429_130746_51.xml,MINVIDAS3,minn-vidas3,387,cbzA589901,SLM,Salmonella,387589901,R2,NEGATIVE,SLM
300,MINVIDAS3_190429_130746_51.xml,MINVIDAS3,minn-vidas3,387,cbzA589901,SLM,Salmonella,387589901,R1,0.03,ODSLM
614,MINVIDAS3_190429_130746_52.xml,MINVIDAS3,minn-vidas3,387,ccaA589908,SLM,Salmonella,387589908,R2,NEGATIVE,SLM
301,MINVIDAS3_190429_130746_52.xml,MINVIDAS3,minn-vidas3,387,ccaA589908,SLM,Salmonella,387589908,R1,0.03,ODSLM
615,MINVIDAS3_190429_130746_53.xml,MINVIDAS3,minn-vidas3,387,ccbA589916,SLM,Salmonella,387589916,R2,NEGATIVE,SLM
302,MINVIDAS3_190429_130746_53.xml,MINVIDAS3,minn-vidas3,387,ccbA589916,SLM,Salmonella,387589916,R1,0.03,ODSLM
616,MINVIDAS3_190429_130746_54.xml,MINVIDAS3,minn-vidas3,387,cccA589922,SLM,Salmonella,387589922,R2,NEGATIVE,SLM
303,MINVIDAS3_190429_130746_54.xml,MINVIDAS3,minn-vidas3,387,cccA589922,SLM,Salmonella,387589922,R1,0.03,ODSLM


In [98]:
dfResult = dfResult.rename(index=str, 
                           columns={'sample_number': 'Sample.Sample_Number', 'value': 'Result.Entry','instrumentName':'Result.Instrument'})

In [99]:
dfResult['Test.Analysis'] = 'VIDAS'
dfResult['Result.Entered_By'] = 'VIDAS_IMPORTER'

In [100]:
dfResult

,filename,Result.Instrument,operatorName,patientIdentifier,specimenIdentifier,testIdentifier,testName,Sample.Sample_Number,variable,Result.Entry,Result.Name,Test.Analysis,Result.Entered_By
612,MINVIDAS3_190429_130746_50.xml,MINVIDAS3,minn-vidas3,387,cbyA589896,SLM,Salmonella,387589896,R2,NEGATIVE,SLM,VIDAS,VIDAS_IMPORTER
299,MINVIDAS3_190429_130746_50.xml,MINVIDAS3,minn-vidas3,387,cbyA589896,SLM,Salmonella,387589896,R1,0.03,ODSLM,VIDAS,VIDAS_IMPORTER
613,MINVIDAS3_190429_130746_51.xml,MINVIDAS3,minn-vidas3,387,cbzA589901,SLM,Salmonella,387589901,R2,NEGATIVE,SLM,VIDAS,VIDAS_IMPORTER
300,MINVIDAS3_190429_130746_51.xml,MINVIDAS3,minn-vidas3,387,cbzA589901,SLM,Salmonella,387589901,R1,0.03,ODSLM,VIDAS,VIDAS_IMPORTER
614,MINVIDAS3_190429_130746_52.xml,MINVIDAS3,minn-vidas3,387,ccaA589908,SLM,Salmonella,387589908,R2,NEGATIVE,SLM,VIDAS,VIDAS_IMPORTER
301,MINVIDAS3_190429_130746_52.xml,MINVIDAS3,minn-vidas3,387,ccaA589908,SLM,Salmonella,387589908,R1,0.03,ODSLM,VIDAS,VIDAS_IMPORTER
615,MINVIDAS3_190429_130746_53.xml,MINVIDAS3,minn-vidas3,387,ccbA589916,SLM,Salmonella,387589916,R2,NEGATIVE,SLM,VIDAS,VIDAS_IMPORTER
302,MINVIDAS3_190429_130746_53.xml,MINVIDAS3,minn-vidas3,387,ccbA589916,SLM,Salmonella,387589916,R1,0.03,ODSLM,VIDAS,VIDAS_IMPORTER
616,MINVIDAS3_190429_130746_54.xml,MINVIDAS3,minn-vidas3,387,cccA589922,SLM,Salmonella,387589922,R2,NEGATIVE,SLM,VIDAS,VIDAS_IMPORTER
303,MINVIDAS3_190429_130746_54.xml,MINVIDAS3,minn-vidas3,387,cccA589922,SLM,Salmonella,387589922,R1,0.03,ODSLM,VIDAS,VIDAS_IMPORTER


In [101]:
dfResult = dfResult[colsOrder]

In [102]:
dfResult

,Sample.Sample_Number,Test.Analysis,Result.Name,Result.Entry,Result.Entered_By,Result.Instrument
612,387589896,VIDAS,SLM,NEGATIVE,VIDAS_IMPORTER,MINVIDAS3
299,387589896,VIDAS,ODSLM,0.03,VIDAS_IMPORTER,MINVIDAS3
613,387589901,VIDAS,SLM,NEGATIVE,VIDAS_IMPORTER,MINVIDAS3
300,387589901,VIDAS,ODSLM,0.03,VIDAS_IMPORTER,MINVIDAS3
614,387589908,VIDAS,SLM,NEGATIVE,VIDAS_IMPORTER,MINVIDAS3
301,387589908,VIDAS,ODSLM,0.03,VIDAS_IMPORTER,MINVIDAS3
615,387589916,VIDAS,SLM,NEGATIVE,VIDAS_IMPORTER,MINVIDAS3
302,387589916,VIDAS,ODSLM,0.03,VIDAS_IMPORTER,MINVIDAS3
616,387589922,VIDAS,SLM,NEGATIVE,VIDAS_IMPORTER,MINVIDAS3
303,387589922,VIDAS,ODSLM,0.03,VIDAS_IMPORTER,MINVIDAS3
